### import
- 솔림, 신탁 따로 업로드결과 파일 만들어서 읽기

In [1]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

# dtype 정의 (사이트 파일은 읽지 말자. 할 게 너무 많음)
# 나우리 타입 반영
nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
# # 신복 사이트 타입 반영
# nauri_dtype = {'채무자키' : str, '계좌키' : str, '보증인키':str, '신용회복키' : str, '심의차수' : str, '조정전이율' : str, '조정후이율' : str, '총상환기간' : str, '유예기간' : str, '원금균등상환기간' : str, '원리균등상환기간' : str, '이자상환기간' : str, '납입회차' : str, '연체기간' : str, '원금균등시작회차' : str, '원금균등종료회차' : str, '원리균등시작회차' : str, '원리균등종료회차' : str, '이자상환시작회차' : str, '이자상환종료회차' : str, '감면율' : str, '주채무자주민번호' : str, '시작회차1' : str, '종료회차1' : str, '채무조정1적용이율1' : str, '채무조정2적용이율1' : str, '시작회차2' : str, '종료회차2' : str, '채무조정1적용이율2' : str, '채무조정2적용이율2' : str, '시작회차3' : str, '종료회차3' : str, '채무조정1적용이율3' : str, '채무조정2적용이율3' : str, '시작회차4' : str, '종료회차4' : str, '채무조정1적용이율4' : str, '채무조정2적용이율4' : str, '시작회차5' : str, '종료회차5' : str, '채무조정1적용이율5' : str, '채무조정2적용이율5' : str, '시작회차6' : str, '종료회차6' : str, '채무조정1적용이율6' : str, '채무조정2적용이율6' : str, '시작회차7' : str, '종료회차7' : str, '채무조정1적용이율7' : str, '채무조정2적용이율7' : str, '시작회차8' : str, '종료회차8' : str, '채무조정1적용이율8' : str, '채무조정2적용이율8' : str, '시작회차9' : str, '종료회차9' : str, '채무조정1적용이율9' : str, '채무조정2적용이율9' : str, '시작회차10' : str, '종료회차10' : str, '채무조정1적용이율10' : str, '채무조정2적용이율10' : str, '재조정횟수' : str, '수정조정횟수' : str, '일시납감면율' : str, '거치기간' : str, '체증전구간의시작회차' : str, '체증전구간의종료회차' : str, '체증전구간의기준기간' : str, '변제금수취계좌' : str, '담보권실행유예기간':str
# }
deposit_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '담당자키':str, '일시납감면율':str, '계좌번호':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}

In [2]:
# 계좌별 진행상황 파일 읽기
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"
########################################################
basedate = "240509"
fn_솔림 = basedate + "_계좌별 진행상황 조회_솔림_업로드결과.xlsx"
fn_신탁 = basedate + "_계좌별 진행상황 조회_신탁_업로드결과.xlsx"
########################################################

# 계좌별 진행상황 조회 파일 읽기
솔림_ori = pd.read_excel(join(path_base, fn_솔림), dtype=nauri_dtype).fillna("")
신탁_ori = pd.read_excel(join(path_base, fn_신탁), dtype=nauri_dtype).fillna("")

In [3]:
# 풀 및 신탁계좌키 읽기
#######################################################################################
keys = pd.read_excel(r"D:\3.자산\전산수정 LOG\240509_계좌_P15_신탁_리파 담보제공처.xlsx", dtype=str)

풀계좌키 = keys.계좌키
신탁계좌키 = keys.query('리파이낸싱.str.contains("신복", na=False)').계좌키
#######################################################################################
print(f'풀계좌수 : {len(풀계좌키)}, 신탁계좌수 : {len(신탁계좌키)}')


풀계좌수 : 2731, 신탁계좌수 : 1014


In [4]:
# 원본 복사
솔림 = 솔림_ori.copy()
신탁 = 신탁_ori.copy()

In [5]:
# 현재채권사 열 추가
솔림.insert(0, "현재채권사", "솔림")
신탁.insert(0, "현재채권사", "신탁")

# 병합
raw = pd.concat([솔림, 신탁], axis=0, ignore_index=True)

# 불필요한 열 삭제
raw.drop(columns=["채무자명","채무상태", "현재채권구분","등록파일","수정파일", "삭제여부","등록일자","등록유저","수정일자","수정유저","확인결과","저장결과"], axis=1, inplace=True)

# 새로운 열을 추가 : insert 여러번 쓰니 성능 경고나와서
진행구분 = pd.Series([None] * len(raw), name='진행구분')
반영OPB = pd.Series([None] * len(raw), name='반영OPB')
상환구분 = pd.Series([None] * len(raw), name='상환구분')

position = 5
raw = pd.concat([raw.iloc[:, :position], 진행구분, 반영OPB, 상환구분, raw.iloc[:, position:]], axis=1)

In [6]:
# 주민 계좌 중복건 : 전산에는 중복건 있더라도 신복위에서 받은 데이터에는 없는것이 당연
raw[raw.duplicated(subset=["신청인주민번호", "계좌번호"], keep=False)]

# 진행구분열 작성-------------------------------
x = raw.신청인진행상태
y = raw.계좌진행상태내용
z = raw.계좌번호

conds = [
    (x == '정상이행') & (y == '채무조정포함'),
    (y == "채무조정제외"),
    (x.str.contains(r"완제|상환") | y.str.contains(r"완제|상환") | z.str.contains(r"완제|상환")),
    (z == "채권신고전"),
    (x.str.contains(r"심사반송|중도실효|합의서체결포기")),
]
values = ["확정", "채무조정제외","완제","채권신고전","실효"]
default_value = "미확정"

raw["진행구분"] = np.select(conds, values, default_value)

# 반영OPB - 확정미확정 건-------------------------------
x = raw.진행구분
# 확정시 우선순위
a = raw.상환후잔액
b = raw.원금균등채무액
c= raw.조정후원금
d = raw.조정전원금 + raw.조정전비용   # 조정정비용 추가 2024-04-03

# 미확정-조정전원금0 은 이자채권으로 그냥 0이 맞다.
conds = [(x == "미확정"), (x =="확정")&(a!=0), b!=0, c!=0, d!=0]

# 조건에 따라 값을 설정
values = [d, a, b, c, d]

# 기본값 설정
default_value = np.nan

# 반영OPB 열 설정
raw["반영OPB"] = np.select(conds, values, default_value)


# 상환구분 작성 - 확정미확정 건-------------------------------
x = raw.상환방식

conds = [x.str.contains("원금균등"), x.str.contains("원리금균등")] #체증식(원금균등)이란게 있네..

# 조건에 따라 값을 설정
values = ["개인","프리"]

# 기본값 설정
default_value = "" # 미확정

# 반영OPB 열 설정
raw["상환구분"] = np.select(conds, values, default_value)

In [15]:
최종 = raw.query('진행구분 in ["확정","미확정","실효","완제"]').copy()
최종.drop(최종[최종.채무자키=='전산미등록건'].index, inplace=True)

# 확인
print('계좌키 없는 것 진행구분')
최종.query('계좌키==""')["진행구분"].value_counts()

계좌키 없는 것 진행구분


진행구분
완제    62
실효    18
Name: count, dtype: int64

In [16]:
# 신탁건만
신탁최종 = 최종.query('계좌키.isin(@신탁계좌키)')
count_신탁최종_차주 = len(신탁최종.query("채무구분=='주채무'"))
print(f'신탁최종 건수 : {len(신탁최종)}, 신탁최종,차주만 : {count_신탁최종_차주}, 신탁건수 : {len(신탁계좌키)}')
if count_신탁최종_차주 != len(신탁계좌키) : 
    # [중요check] 신탁계약했는데 신복이 아닌 경우
    display(신탁계좌키[~신탁계좌키.isin(최종.계좌키)])
else : print("건수 이상 없음")
    

# 채권자변경 넣기
신탁최종.insert(1,"채권자변경", "")
신탁최종.loc[신탁최종.query('현재채권사=="솔림" and ~진행구분.isin(["실효","완제"])').index,["채권자변경"]] = "솔림to신탁" # 양도
신탁최종.loc[신탁최종.query('현재채권사=="신탁" and ~진행구분.isin(["실효","완제"])').index, ["채권자변경"]] = "유지"
신탁최종.loc[신탁최종.query('현재채권사=="신탁" and 진행구분.isin(["실효","완제"])').index, ["채권자변경"]] = "신탁to솔림_기준일이후실효완제" # 양수
신탁최종.loc[신탁최종.query('현재채권사=="솔림" and 진행구분.isin(["실효","완제"])').index, ["채권자변경"]] = "솔림유지_기준일이후실효완제"

# 신탁해지건, 신탁to솔림
신탁해지건 = 최종.query('현재채권사=="신탁" and 계좌키.isin(@풀계좌키) and ~계좌키.isin(@신탁계좌키)') # 양수
신탁해지건.insert(1, "채권자변경", "신탁to솔림_신탁해지")

신탁최종 건수 : 1014, 신탁최종,차주만 : 1014, 신탁건수 : 1014
건수 이상 없음


In [22]:
# 채권자 일괄변경
솔림to신탁 = 신탁최종.query('채권자변경=="솔림to신탁"')[["신청인주민번호","계좌번호"]]
신탁to솔림 = 신탁최종.query('채권자변경=="신탁to솔림_기준일이후실효완제"')[["신청인주민번호","계좌번호"]]
신탁to솔림 = pd.concat([신탁to솔림, 신탁해지건[["신청인주민번호", "계좌번호"]]], axis=0)

# 솔림to신탁
솔림to신탁["신청일"] = "20" + basedate
솔림to신탁["신청인주민번호"] = 솔림to신탁.신청인주민번호.str.replace("-", "")
솔림to신탁["양도기관지로코드"] = "GB21146" # 솔림
솔림to신탁["변경구분코드"] = "630201"
솔림to신탁["양수기관기관코드"] = "GB22770" # 신탁
솔림to신탁["양수기관지로코드"] = "GB22770"
솔림to신탁 = 솔림to신탁[["신청일", "신청인주민번호", "양도기관지로코드", "변경구분코드", "양수기관기관코드", "양수기관지로코드", "계좌번호"]]

# 신탁to솔림
신탁to솔림["신청일"] = "20" + basedate
신탁to솔림["신청인주민번호"] = 신탁to솔림.신청인주민번호.str.replace("-", "")
신탁to솔림["양도기관지로코드"] = "GB22770" # 신탁
신탁to솔림["변경구분코드"] = "630201"
신탁to솔림["양수기관기관코드"] = "GB21146" # 솔림
신탁to솔림["양수기관지로코드"] = "GB21146"
신탁to솔림 = 신탁to솔림[["신청일", "신청인주민번호", "양도기관지로코드", "변경구분코드", "양수기관기관코드", "양수기관지로코드", "계좌번호"]]

In [23]:
# 파일저장 : 업로드용csv
솔림to신탁.to_csv(join(path_base, "채권자 일괄변경_카카오_양도(솔림to신탁).csv"), header=False, index=False, encoding='euc-kr')
신탁to솔림.to_csv(join(path_base, "채권자 일괄변경_카카오_양수(신탁to솔림).csv"), header=False, index=False, encoding='euc-kr')

In [18]:
# 파일저장 : 확인용 >> 비식별처리. 채권자변경파일 만든 후에 해야 함.
신탁최종["신청인주민번호"] = 신탁최종["신청인주민번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
신탁해지건["신청인주민번호"] = 신탁해지건["신청인주민번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))

fn확인용 = join(path_base,"확인용_계좌별 진행상황.xlsx")


# 엑셀 파일로 저장
if not os.path.exists(fn확인용):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(fn확인용, engine='openpyxl', **param) as writer :
    신탁최종.to_excel(writer, sheet_name='신탁계약건', index=False)
    신탁해지건.to_excel(writer, sheet_name='신탁해지건', index=False)